In [20]:

pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
     |████████████████████████████████| 7.0 MB 18.6 MB/s eta 0:00:01
     |████████████████████████████████| 7.8 MB 19.0 MB/s eta 0:00:01
     |████████████████████████████████| 224 kB 19.4 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install numpy pandas transformers torch sklearn


Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
     |████████████████▏               | 312.8 MB 126.4 MB/s eta 0:00:03    | 212.2 MB 110.7 MB/s eta 0:00:04

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |███████████████████████████████▉| 615.9 MB 46.5 MB/s eta 0:00:011█████████████▌| 610.2 MB 46.5 MB/s eta 0:00:01

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 619.9 MB 2.0 kB/s 
     |████████████████████████████████| 11.8 MB 23.1 MB/s eta 0:00:01
     |████████████████████████████    | 278.2 MB 89.3 MB/s eta 0:00:01     |▊                               | 6.7 MB 24.1 MB/s eta 0:00:13     |███████████                     | 108.9 MB 97.9 MB/s eta 0:00:03     |███████████▍                    | 113.2 MB 97.9 MB/s eta 0:00:03/s eta 0:00:02

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

# Load dataset (assuming CSV format with 'review_text', 'sentiment' columns)
data = pd.read_csv('movie_reviews.csv')


In [ ]:
#Tokenize the text using BERT's tokenizer:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize(text):
    return tokenizer.encode_plus(
        text,
        max_length=256,
        truncation=True,
        padding='max_length',
        add_special_tokens=True,
        return_tensors='pt'
    )

# Tokenize the reviews
data['tokens'] = data['review_text'].apply(tokenize)


In [ ]:
#Split the data into training and testing sets and create DataLoaders:
X = data['tokens']
y = data['sentiment'].map({'positive': 1, 'negative': 0})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = TensorDataset(
    torch.cat([item['input_ids'] for item in X_train.values], dim=0),
    torch.cat([item['attention_mask'] for item in X_train.values], dim=0),
    torch.tensor(y_train.values)
)

test_dataset = TensorDataset(
    torch.cat([item['input_ids'] for item in X_test.values], dim=0),
    torch.cat([item['attention_mask'] for item in X_test.values], dim=0),
    torch.tensor(y_test.values)
)

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=16)
test_dataloader = DataLoader(test_dataset, batch_size=16)


In [ ]:
#et up the BERT model, optimizer, and learning rate scheduler:
